Load the necessary packages.  Uncomment the first two lines if it is necessary to add or update the packages.

In [1]:
#using Pkg
#Pkg.add(["Convex","GLPKMathProgInterface", "DataFrames"])
using RunOrderOptim, Convex, GLPKMathProgInterface, DataFrames

Load the class information for the event.

In [2]:
df_classes=RunOrderOptim.read_classes() #Default to example classes
first(df_classes,12) # Truncate the view for display purposes
#show(df_classes,allrows=true) # Uncomment to show all the classes


,Class,ClassGroup,BumpClass
,String,String,String⍰
1,SS,Street,C1
2,AS,Street,C1
3,BS,Street,C1
4,CS,Street,C1
5,DS,Street,C1
6,ES,Street,C1
7,FS,Street,C1
8,GS,Street,C1
9,HS,Street,C1


Load the entries for event from MotorSportsReg.com export

In [3]:
df_entries=RunOrderOptim.read_entries() #Default to example event
first(df_entries,12) # Truncate the view for display purposes
#show(df_entries,allrows=true) # Uncomment to show all the classes

,LastName,FirstName,Class,Index,Exempt,Novice,IndexedClass
,String,String,String,String⍰,Bool,Bool,String
1,Abramovich,David,NA,missing,false,true,NA
2,Anderson,Dwight,HS,Z,false,false,Z
3,Anderson,Kerry,STS,P,true,false,P
4,Arntzen,Erik,GS,missing,false,false,GS
5,Bahl,Dave,FS,Z,false,false,Z
6,Barber,Adam,STS,P,false,false,P
7,Baumbach,Drew,ASP,Z,false,false,Z
8,Beck,Robert,NB,missing,false,true,NB
9,Bedell,Jody,SSC,missing,false,false,SSC


In [4]:
prob=RunOrderOptim.setup_problem(df_classes,df_entries; keep_empty=false) #Setup the problem
prob.x

┌ Info: Combining CAMC, CAMS, CAMT, EP, XP, SSC, DSP, STS, STU
└ @ RunOrderOptim H:\Projects\Julia\RunOrderOptim\src\RunOrderOptim.jl:53
┌ Info: Combining CS, FS, GS, HS, SS
└ @ RunOrderOptim H:\Projects\Julia\RunOrderOptim\src\RunOrderOptim.jl:53


Variable of
size: (51, 2)
sign: NoSign()
vexity: AffineVexity()

In [5]:
solve!(prob.problem,GLPKSolverMIP()) #Solve the problem

In [6]:
prob.problem.status #Check the status of the solution

:Optimal

In [7]:
prob.problem.optval #Solution score.  Lower is better

13.0

In [8]:
df=RunOrderOptim.compact_solution(prob)

,RunGroup,Workers,Drivers,Novice
,Int64,Int64,Int64,Int64
1,1,48,51,11
2,2,47,51,10


In [9]:
RunOrderOptim.full_solution(prob)

,RunGroup,Class,ClassGroup,Drivers,Workers
,Int64,String,String,Int64,Int64
1,1,DM,Modified,4,4
2,1,P,Pro,23,10
3,1,DS,Street,10,9
4,1,SMF,Street Modified,5,5
5,1,SMF,Street Modified,5,5
6,1,SSM,Street Modified,4,4
7,1,Novice A-M,Novice,11,11
8,2,CAMC,Classic American,2,2
9,2,CAMS,Classic American,1,1


In [10]:
show(RunOrderOptim.full_solution(prob), allrows=true)

27×5 DataFrame
│ Row │ RunGroup │ Class      │ ClassGroup       │ Drivers │ Workers │
│     │ Int64    │ String     │ String           │ Int64   │ Int64   │
├─────┼──────────┼────────────┼──────────────────┼─────────┼─────────┤
│ 1   │ 1        │ DM         │ Modified         │ 4       │ 4       │
│ 2   │ 1        │ P          │ Pro              │ 23      │ 10      │
│ 3   │ 1        │ DS         │ Street           │ 10      │ 9       │
│ 4   │ 1        │ SMF        │ Street Modified  │ 5       │ 5       │
│ 5   │ 1        │ SMF        │ Street Modified  │ 5       │ 5       │
│ 6   │ 1        │ SSM        │ Street Modified  │ 4       │ 4       │
│ 7   │ 1        │ Novice A-M │ Novice           │ 11      │ 11      │
│ 8   │ 2        │ CAMC       │ Classic American │ 2       │ 2       │
│ 9   │ 2        │ CAMS       │ Classic American │ 1       │ 1       │
│ 10  │ 2        │ CAMT       │ Classic American │ 1       │ 0       │
│ 11  │ 2        │ X          │ Exhibition       │ 1       │ 1